# BluWave Challenge

The goal of this challenge is to determine one or more outputs from 4 wind turbines are stationary.

In [52]:
# import modules

import pandas as pd
import numpy as np
import os
import sys
import random
import copy
import cPickle as pickle
import datetime
import fancyimpute as fi

import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
import colorlover as cl
import matplotlib.pyplot as plt 

init_notebook_mode(connected=True)

%reload_ext autoreload
%autoreload 2

pd.options.display.float_format = '{:,.4f}'.format

ImportError: No module named fancyimpute

In [2]:
# Load data

data = pd.read_excel("Data.xlsx", sheet_name=None).values()[0]

In [3]:
pw_cols = ['PW1', 'PW2', 'PW3', 'PW4']

## Explore/Clean Data Set

First we take a look at the size, column types and the first couple rows of the dataframe, shown below.  The wind turbine outputs are the columns PW1, PW2, PW3 and PW4. There is a TimeStamp column which contains the epoch timestamps (number of seconds since Jan 1, 1970) for each sample. 

In [20]:
data.shape

(106560, 15)

In [103]:
data.dtypes

TimeStamp              float64
Date                    object
Time                    object
Wind Speed             float64
Wind Dir               float64
Temp                   float64
Unnamed: 6             float64
Unnamed: 7              object
Unnamed: 8              object
PW1                     object
PW2                     object
PW3                     object
PW4                     object
Unnamed: 13             object
 Wind Farm MW Value    float64
dtype: object

In [8]:
data.head()

,TimeStamp,Date,Time,Wind Speed,Wind Dir,Temp,Unnamed: 6,Unnamed: 7,Unnamed: 8,PW1,PW2,PW3,PW4,Unnamed: 13,Wind Farm MW Value
0,"1,501,545,600.0000","""Aug 01/17""",00:00:00,8.6741,211.8667,20.0000,nan,NaN,NaN,"1,106.2140","1,071.4309","1,165.2189","1,079.5228",NaN,"4,407.5907"
1,"1,501,545,900.0000","""Aug 01/17""",00:05:00,8.2880,213.7232,20.0000,nan,NaN,NaN,"1,023.2526",920.8148,"1,126.9757",955.0125,NaN,"3,979.1336"
2,"1,501,546,200.0000","""Aug 01/17""",00:10:00,8.5543,216.4733,20.0000,nan,NaN,NaN,"1,066.6329","1,173.6629","1,116.1848",950.9738,NaN,"4,312.0781"
3,"1,501,546,500.0000","""Aug 01/17""",00:15:00,8.5694,218.5832,20.0000,nan,NaN,NaN,"1,067.4646","1,043.5497","1,171.3774","1,153.1642",NaN,"4,359.9587"
4,"1,501,546,800.0000","""Aug 01/17""",00:20:00,8.7656,218.0367,20.0000,nan,NaN,NaN,"1,092.3378","1,058.1656","1,197.3882","1,243.6106",NaN,"4,602.9744"


In terms of time ranges, the samples start and end at the following dates:

In [4]:
'start: ' + data['Date'].iloc[0] + ' end: ' + data['Date'].iloc[-1]

u'start:  "Aug 01/17" end:  "Aug 05/18"'

So there's roughly a year's worth of data. In addition the time interval between each pair of successive points appears to be 5 minutes.

Its strange, however that the PW columns are object types since they appear to be floats. The unique values for the columns are shown below as well as the number of null values

In [110]:
for w in pw_cols:
    print np.unique(data[w])

[-47.165427 -40.573715 -40.265819 ..., 3008.55566 3011.689958 u' ']
[-51.420665 -48.12117 -46.364245 ..., 3007.130397 3009.956445 u' ']
[-46.234508 -45.469031 -44.202472 ..., 3008.425407 3008.767463 u' ']
[-55.90785 -49.78317 -49.744794 ..., 3007.361389 3007.539973 u' ']


In [46]:
data.isnull().sum()

TimeStamp                 11
Date                       0
Time                       0
Wind Speed                11
Wind Dir                  11
Temp                      11
Unnamed: 6             96205
Unnamed: 7             96205
Unnamed: 8             96205
PW1                       11
PW2                       11
PW3                       11
PW4                       11
Unnamed: 13            96205
 Wind Farm MW Value       11
dtype: int64

There are strings mixed in with the floats and each turbine has 11 points of missing data. Combining NaN and string values, we get the following missing values for each turbine:

In [111]:
data.isnull().sum() + (data == ' ').sum()

TimeStamp                  11
Date                        0
Time                        0
Wind Speed                 11
Wind Dir                   11
Temp                       11
Unnamed: 6              96205
Unnamed: 7             106560
Unnamed: 8             106560
PW1                      1778
PW2                      1830
PW3                      1143
PW4                      1133
Unnamed: 13            106560
 Wind Farm MW Value        11
dtype: int64

These 1500 or so missing values for each turbine represents about 1.4% of the data.  This isn't a huge amount but could become a reocurring issue.

Taking a look at the groups of missing values, we get the following stats for turbine 1:

In [4]:
# Create date time objects of the date and time columns, for easier time differencing.
data['date_time'] = data.apply(lambda x: datetime.datetime.strptime(x['Date'].split('"')[1] + x['Time'], 
                                                                    '%b %d/%y %H:%M:%S'), axis = 1)

In [5]:

n = ((data == ' ') | (data.isnull()))
miss_inds = np.where(np.diff(n['PW1']))[0]
miss_inds = zip(miss_inds[::2], miss_inds[1::2])
pd.DataFrame([{'st_datetime': data['date_time'].iloc[x[0]],
 'duration': (data['date_time'].iloc[x[1]] - data['date_time'].iloc[x[0]])}
               for x in miss_inds])


,duration,st_datetime
0,00:50:00,2017-08-06 14:15:00
1,09:20:00,2017-08-20 06:45:00
2,07:40:00,2017-08-21 10:00:00
3,00:20:00,2017-08-21 17:50:00
4,01:50:00,2017-08-21 18:45:00
5,01:30:00,2017-08-24 09:05:00
6,00:05:00,2017-08-24 11:20:00
7,01:00:00,2017-08-30 13:25:00
8,01:00:00,2017-09-01 13:25:00
9,01:40:00,2017-09-27 11:00:00


The duration of the 'blackouts' range widely, from a couple minutes to many hours.  Many occur in January and August, but its difficult to say if this is a pattern.  

Next we look for blackouts where all turbines are involved:

In [42]:
# Get start and end indices of missing groups
miss_inds_all = np.where(n.loc[:, pw_cols].all(1))[0]
inds1 = np.append(miss_inds_all[np.where(np.diff(miss_inds_all) > 1)[0]], miss_inds_all[-1])
inds0 = np.append(miss_inds_all[0], miss_inds_all[np.where(np.diff(miss_inds_all) > 1)[0] + 1])

# Print info about groups
pd.DataFrame([{'st_datetime': data['date_time'].iloc[x[0]],
 'duration': (data['date_time'].iloc[x[1]] - data['date_time'].iloc[x[0]])}
               for x in zip(inds0, inds1)])

,duration,st_datetime
0,09:15:00,2017-08-20 06:50:00
1,07:35:00,2017-08-21 10:05:00
2,00:15:00,2017-08-21 17:55:00
3,01:45:00,2017-08-21 18:50:00
4,01:25:00,2017-08-24 09:10:00
5,00:00:00,2017-08-24 11:25:00
6,01:35:00,2017-09-27 11:05:00
7,15:05:00,2018-01-04 17:10:00
8,01:25:00,2018-01-19 02:15:00
9,00:10:00,2018-01-19 04:00:00


In [46]:
print ' % of data where all turbine data is missing: ' + str(100 * float(len(miss_inds_all)) / data.shape[0])

 % of data where all turbine data is missing: 0.682244744745


Luckily, only a small proportion of the data has all wind turbine values missing.  Looking at a couple samples of these samples (see below), all other features appear to be constant values or missing altogether.  There's isn't any other information to go on. So we'll delete these rows.

In [49]:
data.loc[miss_inds_all,:]

,TimeStamp,Date,Time,Wind Speed,Wind Dir,Temp,Unnamed: 6,Unnamed: 7,Unnamed: 8,PW1,PW2,PW3,PW4,Unnamed: 13,Wind Farm MW Value,date_time
5554,"1,503,211,800.0000","""Aug 20/17""",06:50:00,5.0000,181.0000,19.0000,nan,NaN,NaN,,,,,NaN,904.0000,2017-08-20 06:50:00
5555,"1,503,212,100.0000","""Aug 20/17""",06:55:00,5.0000,181.0000,19.0000,nan,NaN,NaN,,,,,NaN,904.0000,2017-08-20 06:55:00
5556,"1,503,212,400.0000","""Aug 20/17""",07:00:00,5.0000,181.0000,19.0000,nan,NaN,NaN,,,,,NaN,904.0000,2017-08-20 07:00:00
5557,"1,503,212,700.0000","""Aug 20/17""",07:05:00,5.0000,181.0000,19.0000,nan,NaN,NaN,,,,,NaN,904.0000,2017-08-20 07:05:00
5558,"1,503,213,000.0000","""Aug 20/17""",07:10:00,5.0000,181.0000,19.0000,nan,NaN,NaN,,,,,NaN,904.0000,2017-08-20 07:10:00
5559,"1,503,213,300.0000","""Aug 20/17""",07:15:00,5.0000,181.0000,19.0000,nan,NaN,NaN,,,,,NaN,904.0000,2017-08-20 07:15:00
5560,"1,503,213,600.0000","""Aug 20/17""",07:20:00,5.0000,181.0000,19.0000,nan,NaN,NaN,,,,,NaN,904.0000,2017-08-20 07:20:00
5561,"1,503,213,900.0000","""Aug 20/17""",07:25:00,5.0000,181.0000,19.0000,nan,NaN,NaN,,,,,NaN,904.0000,2017-08-20 07:25:00
5562,"1,503,214,200.0000","""Aug 20/17""",07:30:00,5.0000,181.0000,19.0000,nan,NaN,NaN,,,,,NaN,904.0000,2017-08-20 07:30:00
5563,"1,503,214,500.0000","""Aug 20/17""",07:35:00,5.0000,181.0000,19.0000,nan,NaN,NaN,,,,,NaN,904.0000,2017-08-20 07:35:00


In [50]:
cleaned_data = data.drop(miss_inds_all)

Next we'll look at imputing the remaining missing values.  First we'll plot one of the gaps in PW1 to get a sense of the data.

In [51]:
st_missing = miss_inds[22][0]
end_missing = miss_inds[22][1]

# Get indices of points 12h before gap and 12h after
inds = range(st_missing - (12*12), end_missing + (12*12)) #range(st_missing - (12*12), st_missing + 1) + range(end_missing, end_missing + (12*12))

iplot(go.Figure(data = [go.Scatter(x = data['TimeStamp'].iloc[inds], y = data[w].iloc[inds], 
                                   mode = 'lines', name = w) for w in pw_cols]))

The data from each turbine is very similar, which makes sense if the turbines are close together.  We can make use of this in the inputation, especially in the large gaps of missing data where simple methods like linear interpolation and mean imputation would likely introduce artifacts.  We'll try a KNN method - ind the K rows neighbours in the remaining features and use to impute the missing feature value. We'll also try simple linear interpolation for comparison.

MICE (Multiple Imputation using Chained Equations) based method: Each variable is in-turn estimated from the others using regression.  This package doesn't use multiple imputation, where the process is repeated to improve estimation.


In [54]:
import knnimpute

knnimpute.reference

<module 'knnimpute.reference' from 'C:\ProgramData\Anaconda2\lib\site-packages\knnimpute\reference.pyc'>

In [72]:
rolldata = data.loc[:, pw_cols].iloc[inds, :].rolling(window = 10).mean()
iplot(go.Figure(data = [go.Scatter(x = range(rolldata.shape[0]), y = rolldata[w], 
                                   mode = 'lines', name = w) for w in pw_cols]))

In [62]:
44.0 * 100.0 / (data.shape[0] * 4)

0.010322822822822823

Now we'll plot the output against time for each wind turbine using plotly. Given that there are so many points, we'll pick the first point of every day, just to get a sense of the data. 

In [21]:
pt_interval = int(3600.0 * 24.0 / (5.0 * 60))
for w in ['PW1', 'PW2', 'PW3', 'PW4']:
    iplot(go.Figure(data = [go.Scatter(x = data['TimeStamp'].iloc[::pt_interval], 
                                   y = data[w].iloc[::288], mode = 'lines')],
               layout = go.Layout(yaxis = dict(range = [0, 4000]), title = w)))

At first glance, there doesn't appear to be a trend in these time series.

We can also plot summary statistics over time for each time series.  Here, we plot the mean and variance for each day.

In [100]:
clean_data = data.drop(miss_data_inds).reset_index()
pd.to_numeric(clean_data.loc[:, pw_cols]).groupby(clean_data['Date']).mean()

TypeError: arg must be a list, tuple, 1-d array, or Series

In [108]:
clean_data.loc[clean_data['PW1'] == ' ', :]

,index,TimeStamp,Date,Time,Wind Speed,Wind Dir,Temp,Unnamed: 6,Unnamed: 7,Unnamed: 8,PW1,PW2,PW3,PW4,Unnamed: 13,Wind Farm MW Value
1612,1612,"1,502,029,200.0000","""Aug 06/17""",14:20:00,9.3233,158.0000,21.0000,nan,NaN,NaN,,"1,329.4832","1,574.6273","1,379.9878",NaN,"4,222.1501"
1613,1613,"1,502,029,500.0000","""Aug 06/17""",14:25:00,8.6513,158.0000,21.0000,nan,NaN,NaN,,950.5430,"1,166.8148","1,708.4798",NaN,"3,708.1795"
1614,1614,"1,502,029,800.0000","""Aug 06/17""",14:30:00,8.7041,158.0000,21.0000,nan,NaN,NaN,,875.0353,"1,471.0648","1,372.7089",NaN,"3,688.3795"
1615,1615,"1,502,030,100.0000","""Aug 06/17""",14:35:00,9.0740,158.0000,21.0000,nan,NaN,NaN,,923.7121,"1,464.8160","1,659.3166",NaN,"3,943.1179"
1616,1616,"1,502,030,400.0000","""Aug 06/17""",14:40:00,8.8560,158.0000,21.0000,nan,NaN,NaN,,"1,296.3109","1,223.3349","1,527.4078",NaN,"4,006.2336"
1617,1617,"1,502,030,700.0000","""Aug 06/17""",14:45:00,9.7810,158.0000,21.0000,nan,NaN,NaN,,"1,275.8300","1,533.2222","1,935.9453",NaN,"4,586.2631"
1618,1618,"1,502,031,000.0000","""Aug 06/17""",14:50:00,9.6160,158.0000,21.0000,nan,NaN,NaN,,"1,397.1342","1,247.7170","1,833.4828",NaN,"4,445.2696"
1619,1619,"1,502,031,300.0000","""Aug 06/17""",14:55:00,9.1280,158.0000,21.0000,nan,NaN,NaN,,"1,594.5655","1,220.8153","1,278.5715",NaN,"4,015.7100"
1620,1620,"1,502,031,600.0000","""Aug 06/17""",15:00:00,9.3431,158.0000,21.0000,nan,NaN,NaN,,"1,163.0402","1,712.1366","1,455.7527",NaN,"4,278.8583"
1621,1621,"1,502,031,900.0000","""Aug 06/17""",15:05:00,9.0097,158.0000,21.0000,nan,NaN,NaN,,"1,037.5529","1,372.5808","1,469.8112",NaN,"3,860.3102"


In [45]:
data.isnull().sum()

TimeStamp                 11
Date                       0
Time                       0
Wind Speed                11
Wind Dir                  11
Temp                      11
Unnamed: 6             96205
Unnamed: 7             96205
Unnamed: 8             96205
PW1                       11
PW2                       11
PW3                       11
PW4                       11
Unnamed: 13            96205
 Wind Farm MW Value       11
dtype: int64

In [44]:
day_means = data.groupby('Date').mean()
day_means

,TimeStamp,Wind Speed,Wind Dir,Temp,Unnamed: 6,Wind Farm MW Value
Date,,,,,,
"""Apr 01/18""","1,522,583,850.0000",9.4738,208.1105,4.2292,nan,"6,683.9452"
"""Apr 02/18""","1,522,670,250.0000",5.9605,222.7329,1.3277,nan,"2,007.4516"
"""Apr 03/18""","1,522,756,650.0000",4.2754,180.0378,2.7087,nan,770.6178
"""Apr 04/18""","1,522,843,050.0000",6.7455,130.4163,3.5698,nan,"2,631.5541"
"""Apr 05/18""","1,522,929,450.0000",11.9473,250.7715,1.0854,nan,"9,155.8805"
"""Apr 06/18""","1,523,015,850.0000",10.5332,243.7872,-1.1614,nan,"8,207.8424"
"""Apr 07/18""","1,523,102,250.0000",5.6646,161.5783,1.9786,nan,"1,666.7223"
"""Apr 08/18""","1,523,188,650.0000",5.0219,141.0974,0.4599,nan,"1,682.5584"
"""Apr 09/18""","1,523,275,050.0000",12.0483,326.2896,-0.0067,nan,"1,293.4475"


In [48]:
date_time = map(lambda x: x['Date'].split('"') + x['Time'], )
#datetime.datetime.strptime(data['Date'].iloc[0].split('"')[1], "%b %d/%y")

KeyError: ('Date', u'occurred at index TimeStamp')

In [16]:
data['Date'].iloc[0].split('"')[1]

[u' ', u'Aug 01/17', u'']

In [18]:
data['Time'].iloc[0]

u' 00:00:00'